In [28]:
from ultralytics import YOLO
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(MLP, self).__init__()

        # Thêm nhiều lớp fully connected (Linear)
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, num_classes)

        # Sử dụng hàm kích hoạt ReLU cho các lớp ẩn
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(-1)

    def forward(self, x):
        # Forward pass qua các lớp
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)  # Lớp cuối cùng không có ReLU (cho đầu ra)
        out = self.softmax(out)
        return out
def resize(pose):
    new_arr = np.zeros(204)
    new_arr[:len(pose)] = pose
    return new_arr
labels = ['thich',
 'binh_thuong',
 'hieu',
 'chi_gai',
 'nho',
 'nha',
 'yeu',
 'xin_chao',
 'xin_loi',
 'khong_thich',
 'so',
 'biet',
 'me',
 'hen_gap_lai',
 'khoe',
 'cam_on',
 'anh_trai',
 'tam_biet',
 'to_mo']
input_size = 204 
hidden_size = 64  # lớp ẩn với 64 neuron
num_classes = 19
model_cls = MLP(input_size, hidden_size1=256, hidden_size2=128, hidden_size3=64, num_classes=num_classes)
model_cls.load_state_dict(torch.load('models/model_mlp.pth'))
model_cls.eval()
model_pose = YOLO('yolo11n-pose.pt')
def predict(image):
    result = model_pose.predict(image,verbose = False)
    pose = result[0].keypoints.xyn.cpu().detach().numpy().flatten().astype('float')
    if len(pose)>0:
        pose_tensor = torch.tensor(resize(pose)).unsqueeze(0).float()
        with torch.no_grad():
            outputs = model_cls(pose_tensor)
            prob, predicted_class = outputs.max(1)
            idx = predicted_class.item()
            prob = prob.item()
            name = labels[idx]
            print(name, prob)
    return 'binh thuong'

In [9]:
from ultralytics_v10 import YOLOv10
model = YOLOv10('models/best_v10.pt')
print(model.names)
result = model('image.png',verbose = False)[0]
cls = result.boxes.cls.cpu().detach().numpy()
boxes = result.boxes.xyxy.cpu().detach().numpy().astype('int')


{0: 'cam on', 1: 'hen gap lai', 2: 'khoe', 3: 'khong thich', 4: 'rat vui duoc gap ban', 5: 'so', 6: 'tam biet', 7: 'thich', 8: 'xin chao', 9: 'xin loi'}
ultralytics_v10.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (563, 536)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))


In [ ]:
import cv2
import os

def extract_frames_from_video(video_path, output_folder):
    # Mở video
    cap = cv2.VideoCapture(video_path)
    
    # Kiểm tra nếu không mở được video
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}.")
        return
    
    # Kiểm tra và tạo thư mục nếu không tồn tại
    if not os.path.exists(output_folder):
        try:
            os.makedirs(output_folder)
        except Exception as e:
            print(f"Error creating directory {output_folder}: {e}")
            return
    
    # Kiểm tra số lượng khung hình trong video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video {video_path}: {total_frames}")

    frame_count = 0
    while True:
        ret, frame = cap.read()  # Đọc khung hình từ video
        if not ret:  # Nếu không đọc được khung hình thì thoát vòng lặp
            break

        frame_count += 1
        # Tạo đường dẫn đơn giản cho file ảnh
        output_filename = os.path.join(output_folder, f'frame_{frame_count}.jpg')

        # Lưu khung hình thành file ảnh và kiểm tra lưu có thành công không
        if not cv2.imwrite(output_filename, frame):
            print(f"Error: Could not save frame {frame_count} to {output_filename}")
        else:
            print(f"Frame {frame_count} from {video_path} captured and saved.")

        # Tùy chọn: Nhấn 'q' để thoát sớm nếu muốn
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()  # Giải phóng tài nguyên

def process_videos_in_folder(list_file,folder_path, output_folder):
    # Lặp qua tất cả các file trong thư mục
    for filename in list_file:
        # Kiểm tra nếu file là định dạng video
        if filename.endswith(('.mp4', '.avi', '.mov')):  # Có thể thêm định dạng khác nếu cần
            video_path = os.path.join(folder_path, filename)
            video_output_folder = os.path.join(output_folder, os.path.splitext(filename)[0])  # Tạo thư mục riêng cho từng video
            extract_frames_from_video(video_path, video_output_folder)

def run_in_thread(list_file, folder_path, output_folder):
    process_videos_in_folder(list_file, folder_path, output_folder)

import threading
input_folder = 'E:/dulieuxo_moi/Du_lieu_Xo_moi_nhat/a xô/Tan_tuyen_Quang/2.Cảm ơn'  # Thư mục chứa các video
output_folder = 'E:/dulieuxo_moi/cam_diec_moi_nhat_02/anh_ Ton/ChuaCrop/data_vanxo_01/data_camon'  # Thư mục lưu khung ảnh trích xuất
list_file = os.listdir(input_folder)
split_files = [list_file[i::4] for i in range(4)]

# Tạo và chạy thread
threads = []
for i in range(4):
    t = threading.Thread(target=run_in_thread, args=(split_files[i], input_folder, output_folder))
    threads.append(t)
    t.start()

# Chờ tất cả các thread hoàn thành
for t in threads:
    t.join()
